## Determine if Random Statements are Equivalent

Use a truth table (or intuition, when applicable) to determine whether two "randomly" generated statements are logically equivalent.

In [ ]:
import random

def _display_expression(expr):
    '''Helper to display expressions in nicer logical notation'''

    replacements1 = {
        "not (A and B)": "-A ∨ -B",
        "not (A or B)": "-A ∧ -B",
        "(not A) or (not B)": "A → -B",
        "(not A) and (not B)": "-(-A → B)",
        "(not A) or B": "A → B",
        "(not B) or A": "B → A",
        "B and not A": "-(B → A)",
        "A and not B": "-(A → B)",
    }
    replacements2 = {
        "not (A and B)": "-(A ∧ B)",
        "not (A or B)": "-(A ∨ B)",
        "(not A) or (not B)": "-A ∨ -B",
        "(not A) and (not B)": "-A ∧ -B",
        "(not A) or B": "-A ∨ B",
        "(not B) or A": "A ∨ -B",
        "A and not B": "A ∧ -B",
        "B and not A": "-A ∧ B"
    }

    # get more variation by choosing between conditionals or compound and/or statements
    if random.random() < 0.5:
        return replacements1.get(expr, expr)
    else:
        return replacements2.get(expr, expr)
    
    
def _check_equivalence(expr1, expr2):
    # check logical equivalence by truth table
    equivalent = all(
        eval(expr1, {}, {"A": a, "B": b}) == eval(expr2, {}, {"A": a, "B": b})
        for a in [True, False]
        for b in [True, False]
    )
    return equivalent


def _create_expressions(list, rand=.86):
    '''Randomly(ish) generate two expressions and return them and their equivalence status'''

    # choose an expression at random and rebalance the odds to get more true outcomes
    expr1 = random.choice(list)
    if random.random() < rand:
        expr2 = expr1
    else:
        expr2 = random.choice(list)

    # identify how the expressions should be displayed to the user
    display1 = _display_expression(expr1)
    display2 = _display_expression(expr2)

    # prevent the two expressions from being identical
    while display1 == display2:
        expr2 = random.choice(list)
        display2 = _display_expression(expr2)
    
    equivalent = _check_equivalence(expr1, expr2)

    return display1, display2, equivalent


def _test_prob_of_true(list, trials=1000000, rand=.86):
    '''Helper function to decide how frequently the function should force equivalent statements (to keep the quiz balanced around 50/50)'''

    number_true = 0
    for i in range(trials):
        exp1, exp2, equivalent = _create_expressions(list=list, rand=rand)
        if equivalent == True:
            number_true += 1
    
    print("Percent true: ", number_true/trials)
    

def logical_equivalence_quiz(rounds=5):
    print("🔍 Logical Equivalence Quiz")
    print("Decide whether the two expressions are logically equivalent.")
    print("Type True or False and press Enter.\n")

    expressions = [
        # De Morgan's laws
        "not (A and B)",
        "not (A or B)",
        "(not A) or (not B)",
        "(not A) and (not B)",
        # Conditionals and their negations       
        "(not A) or B",      # A → B
        "(not B) or A",      # B → A
        "A and not B",   # negation of A → B
        "B and not A",   # negation of B → A
    ]

    score = 0
    for i in range(1, rounds + 1):
        display1, display2, equivalent = _create_expressions(list=expressions)

        print(f"Round {i}:")
        print("  Expression 1:", display1)
        print("  Expression 2:", display2)

        guess = input("Are they logically equivalent? (True/False): ").strip()

        # as long as the guess starts with t or f, it's a valid answer and interpretted as true or false respectively
        if guess.lower().startswith("t"):
            guess_bool = True
            if guess_bool == equivalent:
                print(f"✅ Correct! It's {equivalent}.\n")
                score += 1
            else:
                print(f"❌ Incorrect. The correct answer was {equivalent}.\n")
        elif guess.lower().startswith("f"):
            guess_bool = False
            if guess_bool == equivalent:
                print(f"✅ Correct! It's {equivalent}.\n")
                score += 1
            else:
                print(f"❌ Incorrect. The correct answer was {equivalent}.\n")
        else:
            print(f"❌ Invalid response. The correct answer was {equivalent}.\n")



    print(f"🏁 Final score: {score}/{rounds}")
    if score == rounds:
        print("🎉 Perfect! You got them all.")
    elif score >= rounds * 0.8:
        print("👍 Great job! You're close — review the tricky ones.")
    else:
        print("📘 Keep practicing!")


In [ ]:
# Run the quiz
logical_equivalence_quiz(rounds=10)

## Determine if Random Arguments are Valid

Use a truth table (or intutition, when applicable) to determine if each "randomly" generated argument is valid.

In [ ]:
import itertools

def _generate_random_argument(expressions):
    """Generate a random argument with 2 premises and a conclusion."""
    num_premises = 2
    premises = random.sample(expressions, num_premises)
    conclusion = random.choice(expressions)

    # Ensure premises are not identical
    while premises[0] == premises[1]:
        premises[1] = random.choice(expressions)

    # Ensure conclusion is not equivalent to a premise
    while (_check_equivalence(premises[0], conclusion) or _check_equivalence(premises[1], conclusion)):
        conclusion = random.choice(expressions)

    return premises, conclusion


def _is_argument_valid(premises, conclusion):
    """Check argument validity using a truth table."""
    for A, B in itertools.product([True, False], repeat=2):
        premise_values = [eval(p, {}, {"A": A, "B": B}) for p in premises]
        conclusion_value = eval(conclusion, {}, {"A": A, "B": B})

        # If all premises are true and conclusion is false → invalid
        if all(premise_values) and not conclusion_value:
            return False
    return True

def _test_prob_of_valid(list, trials=10000):
    '''Helper function to decide how frequently the function should force invalid arguments (to keep the quiz balanced around 50/50)'''

    number_valid = 0
    for i in range(trials):
        premises, conclusion = _generate_random_argument(list)
        validity = _is_argument_valid(premises, conclusion)
        if validity == True:
            number_valid += 1
    
    print("Percent true: ", number_valid/trials)


def argument_validity_quiz(rounds=5):
    """Interactive quiz where user guesses if a random argument is valid."""
    print("🧠 Argument Validity Quiz")
    print("Decide whether the argument is valid.\n")

    expressions = [
        "not (A and B)", "not (A or B)",
        "(not A) or (not B)", "(not A) and (not B)",
        "(not A) or B", "(not B) or A",
        "A and not B", "B and not A"
    ]

    score = 0
    for i in range(1, rounds + 1):
        premises, conclusion = _generate_random_argument(expressions)
        validity = _is_argument_valid(premises, conclusion)

        print(f"🧪 Argument {i}:")
        for j, premise in enumerate(premises, start=1):
            print(f"  Premise {j}: {_display_expression(premise)}")
        print(f"  Conclusion: {_display_expression(conclusion)}")

        guess = input("Is this argument valid? (True/False): ").strip().lower()
        guess_bool = guess.startswith("t")

        status = "valid" if validity else "invalid"
        if guess_bool == validity:
            print(f"✅ Correct! The argument is {status}.\n")
            score += 1
        else:
            print(f"❌ Incorrect. The argument is {status}.\n")

    print(f"🏁 Final Score: {score}/{rounds}")
    if score == rounds:
        print("🎯 Perfect score!")
    elif score >= rounds * 0.7:
        print("👍 Good work — you're getting the hang of validity!")
    else:
        print("📘 Review truth tables and try again.")


In [ ]:
# Run the quiz
argument_validity_quiz(rounds=10)

### Appendix: Probability Testing

The word _randomly_ is in quotes in the descriptions above because statements that are generated in a truly random way are very unlikely to be equivalent. Steps were taken in these functions to generate quiz with roughly even chances of being true vs. false. 

If you run the functions below, you should see probabilities of approximately 0.50.

In [ ]:
expressions = [
        "not (A and B)", "not (A or B)",
        "(not A) or (not B)", "(not A) and (not B)",
        "(not A) or B", "(not B) or A",
        "A and not B", "B and not A"
    ]

_test_prob_of_true(list=expressions, trials=100000, rand=0.86)

In [ ]:
_test_prob_of_valid(list=expressions, trials=100000)